<a href="https://colab.research.google.com/github/satani99/generative_deep_learning/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import auth

auth.authenticate_user()

In [13]:
# !curl https://sdk.cloud.google.com | bash

In [14]:
# !gcloud init --skip-diagnostics

In [15]:
!gsutil cp gs://quickdraw_dataset/full/numpy_bitmap/camel.npy .


Copying gs://quickdraw_dataset/full/numpy_bitmap/camel.npy...
- [1 files][ 90.8 MiB/ 90.8 MiB]                                                
Operation completed over 1 objects/90.8 MiB.                                     


In [16]:
import torch.nn as nn 
import torch 

In [17]:
# THE DISCRIMINATOR
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    self.conv0 = nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=1)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout()
    self.conv1 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)
    self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
    self.conv3 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
    self.linear = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.dropout(self.relu(self.conv0(x)))
    x = self.dropout(self.relu(self.conv1(x)))
    x = self.dropout(self.relu(self.conv2(x)))
    x = self.dropout(self.relu(self.conv3(x)))
    out = nn.Sigmoid(self.linear(x.view(-1, 2048)))
    return out





In [18]:
# THE GENERATOR 
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    self.linear = nn.Linear(100, 3136)
    self.batchnorm0 = nn.BatchNorm1d(3136)
    self.relu = nn.ReLU()
    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    self.conv0 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.batchnorm1 = nn.BatchNorm2d(128)
    self.conv1 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
    self.batchnorm2 = nn.BatchNorm2d(64)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1) 
    self.batchnorm3 = nn.BatchNorm2d(64)
    self.conv3 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1) 
    self.tanh = nn.Tanh() 

  def forward(self, x):
    x = self.linear(x)
    x = self.relu(self.batchnorm0(x))
    x = self.upsample(x.view(-1, 64, 7, 7))
    x = self.relu(self.batchnorm1(self.conv0(x)))
    x = self.upsample(x)
    x = self.relu(self.batchnorm2(self.conv1(x)))
    x = self.relu(self.batchnorm3(self.conv2(x)))
    out = self.tanh(self.conv3(x)) 
    return out 






In [19]:
netG = Generator() 
netD = Discriminator() 

In [21]:
import torch.optim as optim 

criterion = nn.BCELoss() 

fixed_noise = torch.randn(64, 100, 1, 1)

real_label = 1.
fake_label = 0. 

optimizerD = optim.Adam(netD.parameters(), lr=0.0008)
optimizerG = optim.Adam(netG.parameters(), lr=0.0008)